In [13]:
!pip install apache-airflow-providers-apache-spark

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 212.4 MB 164 kB/s s eta 0:00:01   |███████████▌                    | 76.1 MB 20.5 MB/s eta 0:00:07


     |████████████████████████████████| 198 kB 49.1 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212882708 sha256=2165342cc6fed0249883d72a6904d268380a4f6713da61e71c679d787d793d4f
  Stored in directory: /home/jovyan/.cache/pip/wheels/7a/2d/64/5c1411de860c9cbb22b41fc9d342f70c42e8e2cd69cf9259cd
Successfully built pyspark


In [33]:
from datetime import timedelta, datetime
from airflow import DAG
from airflow.operators.python import PythonOperator,PythonVirtualenvOperator
from airflow.operators.dummy import DummyOperator
from airflow.contrib.operators.spark_submit_operator import SparkSubmitOperator

import load_to_s3_from_nas
import load_to_batchlayer_ticker

ModuleNotFoundError: No module named 'load_to_batchlayer_ticker'

In [8]:
default_args = {
    'owner': 'mccho',
    'depends_on_past': False,
    'email': ['mccho8865@gmail.com'],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
    # 'queue': 'bash_queue',
    # 'pool': 'backfill',
    # 'priority_weight': 10,
    # 'end_date': datetime(2016, 1, 1),
    # 'wait_for_downstream': False,
    # 'dag': dag,
    # 'sla': timedelta(hours=2),
    # 'execution_timeout': timedelta(seconds=300),
    # 'on_failure_callback': some_function,
    # 'on_success_callback': some_other_function,
    # 'on_retry_callback': another_function,
    # 'sla_miss_callback': yet_another_function,
    # 'trigger_rule': 'all_success'
}

In [9]:
dag = DAG(
    'load_bithumb_ticker_hourly',
    default_args=default_args,
    description='A simple tutorial DAG',
    schedule_interval='5 * * * *',
    start_date=datetime(2021, 9, 15),
    tags=['load', 'bithumb'],
)

In [10]:
start = DummyOperator(task_id="start")
end = DummyOperator(task_id="end")

In [11]:
# merge files about hourly ticker and upload to s3 from nas
load_to_s3_from_nas = PythonOperator(
    task_id='load_to_s3_from_nas',
    python_callable=load_to_s3_from_nas.execute,
    op_kwargs={'dt': '{{ ds.strftime("%Y-%m-%d") }}',
               'hh': '{{ ts.strftime("%H")}}'},
    dag=dag
)

In [18]:
# load unique ticker item to warehouse
load_to_batchlayer_ticker = PythonOperator(
    task_id='load_to_batchlayer_ticker',
    python_callable=load_to_batchlayer_ticker.execute,
    op_kwargs={'dt': '{{ ds.strftime("%Y-%m-%d") }}',
               'hh': '{{ ts.strftime("%H")}}'},
    dag=dag
)

'//192.168.0.10/raw/ticker/dt=20210920/hh=00/000000.json'

In [92]:
start >> load_to_s3_from_nas >> load_to_batchlayer_ticker >> end

item count : 3600


In [34]:
import boto3


In [35]:
{'deploy-mode': 'cluster', 'spark_binary': 'spark-submit', 'spark_home': '/usr/local/spark', 'namespace': 'default'}

{'deploy-mode': 'cluster',
 'spark_binary': 'spark-submit',
 'spark_home': '/usr/local/spark',
 'namespace': 'default'}

In [22]:

datetime.strptime('2021-09-15T15:05:00+00:00', '%Y-%m-%dT%H:%M:%S+00:00').strftime('%H')

'15'

In [31]:
def load_to_s3_from_nas(ts):
    kst = pytz.timezone('Asia/Seoul')
    ts = datetime.strptime(ts, '%Y-%m-%dT%H:%M:%S+00:00')
    ts = ts.astimezone(kst)
    dt = ts.strftime("%Y-%m-%d")
    hh = ts.strftime("%H")
    read_dir = f"/raw/ticker/dt={dt}/hh={hh}"
    write_dir = f"/raw/ticker_merged/dt={dt}"
    print(read_dir, write_dir)
#     item_list = []
#     with ftplib.FTP() as ftp:
#         ftp.connect("192.168.0.10", 21)
#         ftp.login()
#         ftp.cwd(read_dir)
#         file_list = ftp.nlst()

#         for file in file_list:
#             item = []
#             ftp.retrlines(f"RETR {file}", item.append)
#             item_list.append("".join(item).replace(" ", ""))
#         print(f"item count : {len(item_list)}")

#     k = Key(coin_bucket)
#     k.key = f"warehouse/raw/ticker/dt={dt}/hh_{hh}.txt"
#     k.set_contents_from_string("\n".join(item_list))
load_to_s3_from_nas('2021-09-15T15:05:00+00:00')

/raw/ticker/dt=2021-09-16/hh=00 /raw/ticker_merged/dt=2021-09-16
